# Building Multi-Agent Systems with Strands Agents SDK (1) Agent as Tools

## Introduction to Multi-Agent Systems

In this sample, we'll explore how to build sophisticated AI systems by combining multiple specialized agents that work together using Claude 3.7 Sonnet on Amazon Bedrock. Multi-agent systems can tackle complex tasks through collaboration, specialization, and coordinated workflows.

We'll cover:
- The benefits of multi-agent architectures
- Patterns for agent collaboration using Strands built-in capability
- Building practical multi-agent systems with Claude 3.7 Sonnet on Amazon Bedrock

## Why Use Multiple Agents?

There are several compelling reasons to design multi-agent systems:

1. **Specialization**: Each agent can excel at specific tasks using targeted tools and prompts
2. **Modularity**: Easier to develop, test, and maintain separate components
3. **Scalability**: Independent agents can work in parallel
4. **Separation of Concerns**: Each agent can handle different aspects of a complex problem
5. **Resilience**: If one agent fails, others can continue operating

Let's start by setting up our environment:

In [1]:
# Install required packages with dependency conflict resolution
import subprocess
import sys
import warnings

def install_package_safe(package, use_no_deps=False):
    """Install package with optional no-deps flag to avoid conflicts."""
    cmd = [sys.executable, "-m", "pip", "install", "-U"]
    if use_no_deps:
        cmd.append("--no-deps")
    cmd.append(package)
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            return True, result.stdout
        else:
            return False, result.stderr
    except Exception as e:
        return False, str(e)

# Install required packages
packages = [
    "strands-agents",
    "strands-agents-tools"
]

print("Installing packages for Claude 3.7 Sonnet multi-agent systems...")
print("Note: Dependency warnings about botocore/awscli can be safely ignored.\n")

for package in packages:
    print(f"Installing {package}...")
    
    # First try normal installation
    success, output = install_package_safe(package)
    if success:
        print(f"✓ Successfully installed {package}")
    else:
        print(f"⚠️  Normal installation had conflicts, trying alternative method...")
        # Try with --no-deps to avoid dependency conflicts
        success, output = install_package_safe(package, use_no_deps=True)
        if success:
            print(f"✓ Successfully installed {package} (bypassed dependency conflicts)")
        else:
            print(f"✗ Failed to install {package}: {output}")

# Suppress dependency warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

print("\n📦 Package installation completed!")
print("\n💡 If you see botocore/awscli dependency warnings above, they can be safely ignored.")
print("   AWS notebook environments handle these conflicts automatically.")

Installing packages for Claude 3.7 Sonnet multi-agent systems...
Note: Dependency warnings about botocore/awscli can be safely ignored.

Installing strands-agents...
✓ Successfully installed strands-agents
Installing strands-agents-tools...
✓ Successfully installed strands-agents-tools

📦 Package installation completed!

💡 If you see botocore/awscli dependency warnings above, they can be safely ignored.
   AWS notebook environments handle these conflicts automatically.


## 🔧 Troubleshooting Installation Issues

### Common Dependency Conflict Warning

If you see an error like:


**This warning can be safely ignored!** Here's why:

- AWS notebook environments manage core AWS functionality automatically
- The newer botocore version is backward compatible
- Strands Agents will work correctly despite the warning
- This is a known issue in AWS managed environments

### Alternative Installation Methods

If you prefer to avoid the warning, try these alternatives:

In [2]:
# Alternative installation methods (run only if needed)

# Method 1: Install with --no-deps to bypass conflicts
# !pip install --no-deps strands-agents strands-agents-tools

# Method 2: Install with --disable-pip-version-check
# !pip install --disable-pip-version-check -U strands-agents strands-agents-tools

# Method 3: Force reinstall
# !pip install --force-reinstall strands-agents strands-agents-tools

print("💡 Uncomment and run one of the methods above only if you're experiencing issues.")
print("   The standard installation above should work fine despite any warnings.")

💡 Uncomment and run one of the methods above only if you're experiencing issues.
   The standard installation above should work fine despite any warnings.


### Verification Test

Run the cell below to verify everything is working correctly:

In [3]:
# Verification test
print("🔍 Testing installation...")

try:
    from strands import Agent
    from strands_tools import calculator, swarm, agent_graph, workflow
    print("✅ All required imports successful!")
    
    # Test basic agent creation (without calling the model)
    test_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="You are a test agent."
    )
    print("✅ Claude 3.7 Sonnet agent creation successful!")
    print("\n🎉 Installation verified - ready to proceed with multi-agent systems!")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   Try running one of the alternative installation methods above.")
except Exception as e:
    print(f"⚠️  Agent creation issue: {e}")
    print("   This might be an AWS credentials issue - we'll address this in the next section.")

🔍 Testing installation...
✅ All required imports successful!
✅ Claude 3.7 Sonnet agent creation successful!

🎉 Installation verified - ready to proceed with multi-agent systems!


In [4]:
# Setup AWS Bedrock configuration
import boto3
import os
from strands import Agent, tool
from strands_tools import calculator

# Configure AWS region for Bedrock
AWS_REGION = "us-east-1"  # Claude 3.7 Sonnet is available in us-east-1
CLAUDE_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Verify AWS credentials are configured
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials is None:
        print("⚠️  AWS credentials not found. Please configure your AWS credentials.")
        print("   You can use: aws configure")
        print("   Or set environment variables: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY")
    else:
        print("✓ AWS credentials configured")
        
    # Test Bedrock access
    bedrock = boto3.client('bedrock', region_name=AWS_REGION)
    print("✓ Amazon Bedrock client initialized")
    
except Exception as e:
    print(f"✗ Error setting up AWS: {e}")
    print("Please ensure you have proper AWS credentials and permissions for Bedrock.")

✓ AWS credentials configured
✓ Amazon Bedrock client initialized


### 🎯 Model Selection Information

We're using **Claude 3.7 Sonnet** which is currently the best available model on Amazon Bedrock for multi-agent systems:

| Model | Model ID | Best For | Cost |
|-------|----------|----------|------|
| **Claude 3.7 Sonnet** | `us.anthropic.claude-3-7-sonnet-20250219-v1:0` | **Multi-agent systems** (recommended) | ~$3/$15 per 1M tokens |
| Claude 3 Opus | `anthropic.claude-3-opus-20240229-v1:0` | Most capable, complex reasoning | ~$15/$75 per 1M tokens |
| Claude 3 Haiku | `anthropic.claude-3-haiku-20240307-v1:0` | Fast, simple tasks | ~$0.25/$1.25 per 1M tokens |

**Why Claude 3.7 Sonnet?**
- ✅ Excellent reasoning for agent coordination
- ✅ Large context window (200K tokens)
- ✅ Superior tool use capabilities
- ✅ Cost-effective for complex multi-agent tasks
- ✅ Proven performance on Amazon Bedrock

## Multi-Agent Collaboration Patterns

Strands Agents provides several powerful tools for multi-agent collaboration:
- Agent as Tools
- Swarm
- Graph
- Workflow

In this jupyter notebook, Let's explore how to implement "Agent as Tools" with Claude 3.7 Sonnet on Amazon Bedrock to create collaboration pattern:

### Manager-Workers Pattern with Agent as Tool

The agent-as-tool approach allows us to create manager-worker hierarchies where one agent can use other agents as tools. This pattern is ideal for creating manager-worker hierarchies:

```
                 ┌─→ Worker Agent A

Manager Agent  ──┼─→ Worker Agent B

                 └─→ Worker Agent C
```

Let's implement a manager-workers pattern using Claude 3.7 Sonnet and the agent_as_tool functionality:

In [5]:
from strands import Agent, tool
from strands_tools import calculator

# Worker Agent A: Data Processor
@tool
def data_processor(query: str) -> str:
    """Process and structure raw data into organized formats."""
    try:
        data_processor_agent = Agent(
            model=CLAUDE_MODEL_ID,
            system_prompt="""You are a data processing specialist powered by Claude 3.7 Sonnet. 
            Your job is to take raw data and transform it into a structured format.
            Extract key metrics and present data in a clean, organized way.
            
            Guidelines:
            - Structure data using tables, lists, or hierarchical formats
            - Identify and highlight key metrics and trends
            - Remove noise and focus on actionable information
            - Provide clear data summaries"""
        )

        response = data_processor_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in data processor: {str(e)}"

# Worker Agent B: Financial Analyst
@tool
def financial_analyst(query: str) -> str:
    """Analyze financial data and provide insights on trends, risks, and opportunities."""
    try:
        financial_analyst_agent = Agent(
            model=CLAUDE_MODEL_ID,
            tools=[calculator],
            system_prompt="""You are a financial analysis specialist powered by Claude 3.7 Sonnet.
            Analyze financial data to identify trends, risks, and opportunities.
            Focus on metrics like revenue, costs, margins, and growth rates.
            
            Guidelines:
            - Calculate key financial ratios and metrics
            - Identify trends and patterns in financial data
            - Assess financial health and performance
            - Highlight risks and opportunities
            - Use the calculator tool for precise calculations
            - Provide actionable financial insights"""
        )

        response = financial_analyst_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in financial analyst: {str(e)}"

# Worker Agent C: Market Researcher
@tool
def market_researcher(query: str) -> str:
    """Analyze market trends, consumer behavior, and competitive landscape."""
    try:
        market_researcher_agent = Agent(
            model=CLAUDE_MODEL_ID,
            system_prompt="""You are a market research specialist powered by Claude 3.7 Sonnet.
            Analyze market trends, consumer behavior, and competitive landscape.
            Focus on identifying market opportunities and threats.
            
            Guidelines:
            - Analyze market size, growth, and trends
            - Assess competitive positioning and dynamics
            - Identify customer segments and behaviors
            - Evaluate market opportunities and threats
            - Provide strategic market insights
            - Consider industry-specific factors"""
        )

        response = market_researcher_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in market researcher: {str(e)}"

# Manager Agent that uses worker agents as tools
manager_agent = Agent(
    model=CLAUDE_MODEL_ID,
    tools=[data_processor, financial_analyst, market_researcher],
    system_prompt="""You are a research manager powered by Claude 3.7 Sonnet, coordinating a team of specialists.
    You have three specialists on your team that you can use as tools:
    
    1. data_processor: Expert at processing and structuring data
    2. financial_analyst: Expert at analyzing financial metrics and calculations
    3. market_researcher: Expert at analyzing market trends and opportunities
    
    When given a complex task:
    1. Break it down into appropriate subtasks
    2. Delegate subtasks to the most suitable specialists
    3. Synthesize their responses into a comprehensive, actionable answer
    4. Provide strategic recommendations based on all specialist inputs
    
    Always coordinate the work effectively and provide a cohesive final analysis."""
)

print("✓ Manager-Worker system with Claude 3.7 Sonnet initialized")

✓ Manager-Worker system with Claude 3.7 Sonnet initialized


In [6]:
# Test our manager-worker system with a business analysis task
complex_query = """
Analyze this quarterly business data and provide strategic recommendations:

Revenue: $2.45M (up 12% YoY)
Gross Margin: 42% (down 3% from last quarter)
Customer Acquisition Cost: $85 (up 10% YoY)
Customer Lifetime Value: $520 (up 5% YoY)
Market Share: 23% (up 2% YoY)
Competitor A growth rate: 15%
Competitor B growth rate: 8%
Industry growth rate: 10%
Product Line A revenue: $1.2M
Product Line B revenue: $0.85M
Product Line C revenue: $0.4M
"""

print("🔄 Running manager-worker analysis...")
response = manager_agent(complex_query)
print("\n📊 Manager-Worker Analysis Results:")
print("=" * 50)
print(response)

🔄 Running manager-worker analysis...
I'll analyze this quarterly business data by breaking it down into appropriate subtasks and delegating to our specialist team. Let me coordinate this analysis for you.

First, I'll have our data processor organize the raw data into structured categories:
Tool #1: data_processor
# Business Data Processing Request

I'd need the actual quarterly business data to organize it. Without specific data, I can show you a structured framework for organizing quarterly business metrics:

## Quarterly Business Performance Framework

### Financial Metrics
| Metric | Q1 | Q2 | Q3 | Q4 | YoY Change |
|--------|----|----|----|----|------------|
| Revenue | - | - | - | - | - |
| Gross Profit | - | - | - | - | - |
| Operating Margin | - | - | - | - | - |
| Net Income | - | - | - | - | - |
| Cash Flow | - | - | - | - | - |
| EBITDA | - | - | - | - | - |

### Customer Metrics
| Metric | Q1 | Q2 | Q3 | Q4 | YoY Change |
|--------|----|----|----|----|------------|
| Custom

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 85/520              │                                                                            │
│  │ Result    │ 0.1634615385        │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Revenue Breakdown by Product Line
Tool #2: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 1.2/2.45            │                                                                            │
│  │ Result    │ 0.4897959184        │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Tool #3: calculator


...


Tool #4: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 0.4/2.45            │                                                                            │
│  │ Result    │ 0.1632653061        │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Financial Analysis Summary

### Profitability Trends
- **Revenue**: $2.45M with 12% YoY growth, indicating strong top-line performance
- **Gross Margin**: 42%, down 3% from last quarter, signaling pressure on profitability
- **Gross Profit**: 
Tool #5: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 2.45 * 0.42         │                                                                            │
│  │ Result    │ 1.029               │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The quarterly gross profit is $1.029M. The 3% decline in gross margin from last quarter is concerning and suggests increasing cost pressures that need addressing.

### CAC/LTV Analysis
- **CAC**: $85 (up 10% YoY)
- **LTV**: $520 (up 5% YoY)
- **LTV:CAC Ratio**: 6.12:1 (industry benchmark typically suggests 3:1 as healthy)
- **LTV Growth vs CAC Growth**: CAC is growing faster than LTV (10% vs 5%), indicating declining acquisition efficiency

### Product Line Performance
- **Product Line A**: $1.2M (49.0% of total revenue)
- **Product Line B**: $0.85M (34.7% of total revenue)
- **Product Line C**: $0.4M (16.3% of total revenue)

## Key Insights & Recommendations

1. **Gross Margin Pressure**:
   - The 3% quarterly decline in gross margin requires immediate attention
   - Review cost structures and pricing strategies, especially for high-volume products
   - Consider targeted price increases on high-margin items to offset cost increases

2. **Customer Acquisition Efficiency**:
   - While 

In [7]:
# Translate the above analysis into Simplified Chinese
complex_query = """
Translate the above analysis into Simplified Chinese
"""

print("🔄 Translating the above manager-worker analysis into Simplified Chinese...")
response = manager_agent(complex_query)
print("\n📊 Manager-Worker Analysis Results in Simplified Chinese:")
print("=" * 50)
print(response)

🔄 Translating the above manager-worker analysis into Simplified Chinese...
I'll translate the comprehensive business analysis into Simplified Chinese for you.

# 季度业务表现分析与战略建议

## 执行摘要

您的业务显示强劲的收入增长，同比增长12%（2.45百万美元），超过行业平均水平（10%），但面临毛利率下降和客户获取成本增加的挑战。虽然您的市场地位保持强劲，拥有23%的市场份额，但竞争对手A的增长率更快，达15%，需要战略关注。产品线A在收入结构中占主导地位，为1.2百万美元（49%）。

## 关键绩效指标与趋势

### 财务表现
- **收入：** 2.45百万美元（同比↑12%）- 健康增长超过行业平均水平
- **毛利率：** 42%（较上季度↓3%）- 令人担忧的下行趋势
- **毛利润：** 本季度1.029百万美元
- **产品组合：**
  - 产品线A：1.2百万美元（49.0%）
  - 产品线B：0.85百万美元（34.7%）
  - 产品线C：0.4百万美元（16.3%）

### 客户指标
- **客户获取成本(CAC)：** 85美元（同比↑10%）- 增长速度超过理想水平
- **客户终身价值(LTV)：** 520美元（同比↑5%）- 增长，但仅为CAC增长率的一半
- **LTV:CAC比率：** 6.12:1 - 仍然健康（基准为3:1），但呈下降趋势

### 竞争格局
- **您的增长率：** 收入12%，市场份额同比增长2%
- **市场份额：** 23%（同比↑2%）
- **竞争对手A增长：** 15% - 超过您和行业水平
- **竞争对手B增长：** 8% - 低于行业表现
- **行业增长：** 10% - 健康的市场条件

## 识别的战略问题

1. **利润压力：** 毛利率季度下降3%表明成本压力增加或价格竞争加剧。

2. **客户获取效率：** CAC增长速度是LTV的两倍（10%对5%），威胁未来盈利能力。

3. **竞争压力：** 尽管市场份额有所增长，但竞争对手A增长更快（15%），可能构成重大威胁。

4. **产品组合平